In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install fastapi uvicorn pyngrok transformers==4.52.4 accelerate python-docx -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 95.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import numpy as np
import pandas as pd
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from fastapi import FastAPI, Request, HTTPException, UploadFile, File
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import FileResponse
import uvicorn
import threading
import time
import socket
from pyngrok import ngrok, conf
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
import tempfile
import base64


LANGUAGES = {
    'python': ['.py'], 'javascript': ['.js', '.jsx'], 'typescript': ['.ts', '.tsx'],
    'java': ['.java'], 'c++': ['.cpp', '.hpp'], 'c': ['.c', '.h'],
    'c#': ['.cs'], 'go': ['.go'], 'rust': ['.rs'], 'php': ['.php']
}

def detect_language(filename):
    """Auto-detect language from file extension"""
    ext = os.path.splitext(filename)[1].lower()
    for lang, extensions in LANGUAGES.items():
        if ext in extensions:
            return lang
    return None




In [4]:
model_name = "mistralai/Mistral-Nemo-Instruct-2407"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.float16, 
    device_map="auto"
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

2025-11-26 06:28:51.989014: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764138532.169661      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764138532.219361      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

model-00003-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [5]:
def generate_text(prompt, max_new_tokens=1200, temperature=0.4, top_p=0.92):
    """Generate text using the model with better error handling"""
    try:
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        input_length = inputs.input_ids.shape[1]
        
        # Add attention mask
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            top_k=50,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1,
            num_return_sequences=1
        )
        
        generated_tokens = outputs[0][input_length:]
        response = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
        
        # If response is empty or too short, return a fallback message
        if not response or len(response) < 10:
            return "I apologize, but I couldn't generate a proper response. Please try rephrasing your question or ask something more specific about the code."
        
        return response
        
    except Exception as e:
        print(f"Error in generate_text: {str(e)}")
        return f"Error generating response: {str(e)}"

In [6]:
def chat_with_code(message, code_content, language):
    """Chat about the uploaded code with improved prompting"""
    if not code_content:
        return "⚠️ Please upload a code file first!"
    
    # Limit code length to avoid token limits
    code_snippet = code_content[:2500] if len(code_content) > 2500 else code_content
    
    # Better structured prompt
    context = f"""<s>[INST] You are an expert {language} programming assistant. Analyze the following code and answer the user's question clearly and concisely.

CODE:
```{language}
{code_snippet}
```

QUESTION: {message}

Provide a detailed, specific answer about the code. Do not say you cannot see the code or that you need more information. Answer based on what you can see.[/INST]

ANSWER: """
    
    # Generate response with retry logic
    max_retries = 2
    for attempt in range(max_retries):
        response = generate_text(context, max_new_tokens=600, temperature=0.5)
        
        # Check if response is meaningful
        if response and len(response.strip()) > 20 and "couldn't generate" not in response.lower():
            return response
        
        if attempt < max_retries - 1:
            print(f"Attempt {attempt + 1} failed, retrying...")
            time.sleep(1)
    
    return "I'm having trouble generating a response. Please try asking your question differently or be more specific."


def create_doc_prompt(code, language):
    """Create strict prompt that forces structured markdown output"""
    return f"""You are a technical documentation writer. Generate comprehensive documentation for this code.

IMPORTANT: Follow this EXACT structure:

# Overview
Write 2-3 sentences explaining what this code does and its main purpose.

# Dependencies
List all imports/libraries and explain what each one is used for:
- **Library Name**: What it's used for
- **Another Library**: Its purpose

# Main Components

## Functions

For each function, document it like this:

### function_name(parameters)
- **Purpose**: What this function does
- **Parameters**: 
  - param1 (type): description
  - param2 (type): description
- **Returns**: What it returns
- **Example**:
```{language}
# Example usage
result = function_name(arg1, arg2)
```

## Classes

For each class, document it like this:

### ClassName
- **Purpose**: What this class does
- **Attributes**:
  - attribute1: description
  - attribute2: description
- **Methods**:
  - method1(): description
  - method2(): description

# Usage Example
```{language}
# Complete working example
# Show how to use the main functionality
```

# Important Notes
- List any limitations
- Best practices
- Common mistakes to avoid

NOW DOCUMENT THIS {language.upper()} CODE:

```{language}
{code}
```

Remember: Be specific and detailed. Include actual examples."""


def create_formatted_doc(documentation, filename, language):
    """Create professionally formatted Word document"""
    doc = Document()
    
    # Title page
    title = doc.add_heading('Code Documentation', 0)
    title.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    subtitle = doc.add_paragraph()
    subtitle_run = subtitle.add_run(f'File: {filename}')
    subtitle_run.bold = True
    subtitle_run.font.size = Pt(14)
    subtitle_run.font.color.rgb = RGBColor(128, 128, 128)
    subtitle.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    lang_para = doc.add_paragraph()
    lang_run = lang_para.add_run(f'Language: {language.upper()}')
    lang_run.font.size = Pt(12)
    lang_run.font.color.rgb = RGBColor(0, 102, 204)
    lang_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    doc.add_paragraph()
    doc.add_page_break()
    
    # Check if documentation is empty or too short
    if not documentation or len(documentation.strip()) < 50:
        doc.add_heading('Error', 1)
        para = doc.add_paragraph()
        para.add_run('Documentation generation failed or produced insufficient content.')
        para.add_run('\n\nPlease try again or check the code file.')
        return doc
    
    # Parse documentation with proper code block handling
    lines = documentation.split('\n')
    in_code_block = False
    code_buffer = []
    
    for line in lines:
        # Check for code block delimiters
        if line.strip().startswith('```'):
            if in_code_block:
                # End of code block - add buffered code
                if code_buffer:
                    code_text = '\n'.join(code_buffer)
                    para = doc.add_paragraph(code_text)
                    para.style = 'Intense Quote'
                    if para.runs:
                        para.runs[0].font.name = 'Courier New'
                        para.runs[0].font.size = Pt(10)
                    code_buffer = []
                in_code_block = False
            else:
                # Start of code block
                in_code_block = True
            continue
        
        # If inside code block, buffer the line
        if in_code_block:
            code_buffer.append(line)
            continue
        
        # Process normal lines
        line = line.strip()
        if not line:
            continue
            
        # Level 1 Headers
        if line.startswith('# ') and not line.startswith('##'):
            heading_text = line.replace('# ', '')
            heading = doc.add_heading(heading_text, 1)
            for run in heading.runs:
                run.font.color.rgb = RGBColor(0, 102, 204)
        
        # Level 2 Headers
        elif line.startswith('## ') and not line.startswith('###'):
            heading_text = line.replace('## ', '')
            heading = doc.add_heading(heading_text, 2)
            for run in heading.runs:
                run.font.color.rgb = RGBColor(51, 153, 255)
                
        # Level 3 Headers
        elif line.startswith('### '):
            heading_text = line.replace('### ', '')
            doc.add_heading(heading_text, 3)
        
        # Bullet points
        elif line.startswith('- ') or line.startswith('* '):
            content = line[2:]
            para = doc.add_paragraph(style='List Bullet')
            para.paragraph_format.left_indent = Inches(0.5)
            
            # Handle bold within bullets
            if '**' in content:
                parts = content.split('**')
                for i, part in enumerate(parts):
                    run = para.add_run(part)
                    if i % 2 == 1:
                        run.bold = True
            else:
                para.add_run(content)
        
        # Bold text in paragraphs
        elif '**' in line:
            para = doc.add_paragraph()
            parts = line.split('**')
            for i, part in enumerate(parts):
                run = para.add_run(part)
                if i % 2 == 1:
                    run.bold = True
        
        # Regular text
        else:
            doc.add_paragraph(line)
    
    # Footer
    doc.add_page_break()
    footer_para = doc.add_paragraph()
    footer_run = footer_para.add_run('Generated by Code-to-Docs Converter\nPowered by Mistral AI')
    footer_run.font.size = Pt(10)
    footer_run.font.color.rgb = RGBColor(128, 128, 128)
    footer_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    return doc


def generate_documentation(code_content, filename, language):
    """Generate full documentation with Word file"""
    if not code_content:
        return "⚠️ Please upload a code file first!", None
    
    try:
        print(f"Starting documentation generation for {filename}...")
        
        # Generate documentation text with better prompt
        prompt = create_doc_prompt(code_content, language)
        print("Generating documentation text...")
        docs = generate_text(prompt, max_new_tokens=1500, temperature=0.3)
        
        print(f"Documentation generated. Length: {len(docs)} characters")
        print(f"Preview: {docs[:200]}...")
        
        # Create Word document
        print("Creating Word document...")
        doc = create_formatted_doc(docs, filename, language)
        
        # Save to temporary file
        with tempfile.NamedTemporaryFile(mode='wb', suffix='.docx', delete=False) as tmp_file:
            doc.save(tmp_file.name)
            word_filename = tmp_file.name
        
        print(f"Word document saved: {word_filename}")
        
        return docs, word_filename
        
    except Exception as e:
        print(f"Error in generate_documentation: {str(e)}")
        import traceback
        traceback.print_exc()
        return f"Error generating documentation: {str(e)}", None


In [ ]:
NGROK_TOKEN = "YOUR_NGROK_TOKEN_HERE"  # Get from https://dashboard.ngrok.com
API_KEY = "YOUR_SECRET_KEY_HERE"  # Change this to your own secret

In [8]:
print("🚀 Setting up FastAPI...")

app = FastAPI(
    title="Code Documentation API",
    description="AI-powered code documentation and analysis",
    version="1.0.0"
)

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

def check_auth(request: Request):
    """Check API key authorization"""
    auth_header = request.headers.get("authorization")
    if not auth_header or auth_header != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401, detail="Unauthorized - Invalid API Key")

@app.get("/")
async def root():
    """Health check endpoint"""
    return {
        "status": "online",
        "message": "Code Documentation API is running",
        "model": model_name,
        "endpoints": ["/generate", "/chat", "/documentation"]
    }

@app.post("/generate")
async def generate(req: Request):
    """Generate text based on prompt - Generic endpoint"""
    check_auth(req)
    
    try:
        data = await req.json()
        prompt = data.get("prompt", "")
        max_length = data.get("max_length", 300)
        
        if not prompt:
            raise HTTPException(status_code=400, detail="Prompt is required")
        
        response_text = generate_text(prompt, max_new_tokens=max_length)
        
        return {
            "response": response_text,
            "prompt_length": len(prompt),
            "response_length": len(response_text)
        }
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Generation error: {str(e)}")

@app.post("/chat")
async def chat(req: Request):
    """Chat about code with validation"""
    check_auth(req)
    
    try:
        data = await req.json()
        message = data.get("message", "")
        code_content = data.get("code_content", "")
        language = data.get("language", "python")
        
        if not message:
            raise HTTPException(status_code=400, detail="Message is required")
        if not code_content:
            raise HTTPException(status_code=400, detail="Code content is required")
        
        print(f"\n{'='*60}")
        print(f"Chat request: {message}")
        print(f"Language: {language}")
        print(f"Code length: {len(code_content)} characters")
        print(f"{'='*60}\n")
        
        response = chat_with_code(message, code_content, language)
        
        # Validate response before sending
        if not response or len(response.strip()) < 10:
            response = "I apologize, but I couldn't generate a meaningful response. Please try rephrasing your question."
        
        print(f"Response generated: {len(response)} characters")
        print(f"Preview: {response[:100]}...")
        
        return {
            "response": response,
            "message": message,
            "language": language
        }
    
    except Exception as e:
        print(f"Error in chat endpoint: {str(e)}")
        import traceback
        traceback.print_exc()
        raise HTTPException(status_code=500, detail=f"Chat error: {str(e)}")
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Chat error: {str(e)}")

@app.post("/documentation")
async def documentation(req: Request):
    """Generate documentation for code"""
    check_auth(req)
    
    try:
        data = await req.json()
        code_content = data.get("code_content", "")
        filename = data.get("filename", "code.py")
        language = data.get("language", "python")
        
        if not code_content:
            raise HTTPException(status_code=400, detail="Code content is required")
        
        print(f"\n{'='*60}")
        print(f"Documentation request received:")
        print(f"Filename: {filename}")
        print(f"Language: {language}")
        print(f"Code length: {len(code_content)} characters")
        print(f"{'='*60}\n")
        
        docs, doc_file = generate_documentation(code_content, filename, language)
        
        if not doc_file:
            raise HTTPException(status_code=500, detail=f"Failed to generate documentation: {docs}")
        
        # Read the file and encode to base64
        with open(doc_file, "rb") as f:
            doc_bytes = f.read()
            doc_base64 = base64.b64encode(doc_bytes).decode('utf-8')
        
        print(f"Documentation generated successfully!")
        print(f"Doc length: {len(docs)} characters")
        print(f"File size: {len(doc_bytes)} bytes")
        
        return {
            "documentation": docs,
            "filename": os.path.basename(doc_file),
            "file_base64": doc_base64,
            "language": language
        }
    
    except Exception as e:
        import traceback
        traceback.print_exc()
        raise HTTPException(status_code=500, detail=f"Documentation error: {str(e)}")

@app.post("/detect-language")
async def detect_lang(req: Request):
    """Detect programming language from filename"""
    check_auth(req)
    
    try:
        data = await req.json()
        filename = data.get("filename", "")
        
        if not filename:
            raise HTTPException(status_code=400, detail="Filename is required")
        
        language = detect_language(filename)
        
        return {
            "filename": filename,
            "language": language,
            "detected": language is not None
        }
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Detection error: {str(e)}")


🚀 Setting up FastAPI...


In [ ]:
print("🌐 Starting server with ngrok...")

def find_free_port():
    """Find an available port"""
    s = socket.socket()
    s.bind(('', 0))
    port = s.getsockname()[1]
    s.close()
    return port

def run_server(port):
    """Run uvicorn server"""
    uvicorn.run(app, host="0.0.0.0", port=port, log_level="info")

# Configure ngrok
port = find_free_port()
conf.get_default().auth_token = NGROK_TOKEN

# Start ngrok tunnel
public_url = ngrok.connect(port).public_url

print("\n" + "="*70)
print("🎉 CODE DOCUMENTATION API IS RUNNING!")
print("="*70)
print(f"📡 Public URL: {public_url}")
print(f"🔑 API Key: {API_KEY}")
print(f"🔌 Local Port: {port}")
print("="*70)
print("\n📋 COPY THESE TO YOUR STREAMLIT APP:")
print(f"   Ngrok URL: {public_url}")
print(f"   API Key: {API_KEY}")
print("="*70)
print("\n🔗 Available Endpoints:")
print(f"   GET  {public_url}/")
print(f"   POST {public_url}/generate")
print(f"   POST {public_url}/chat")
print(f"   POST {public_url}/documentation")
print(f"   POST {public_url}/detect-language")
print("="*70)
print("\n💡 Test your API:")
print(f"   curl '{public_url}/'")
print("="*70)

# Start server in background thread
server_thread = threading.Thread(target=run_server, args=(port,), daemon=True)
server_thread.start()

# Keep the notebook running
print("\n⏳ Server is running... Keep this cell running!")
print("🛑 To stop: Interrupt the kernel\n")

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n🛑 Server stopped!")
    ngrok.disconnect(public_url)

🌐 Starting server with ngrok...
                                                                                                    
🎉 CODE DOCUMENTATION API IS RUNNING!
📡 Public URL: https://jacquelyne-veiniest-gilda.ngrok-free.dev
🔑 API Key: secret123
🔌 Local Port: 43509

📋 COPY THESE TO YOUR STREAMLIT APP:
   Ngrok URL: https://jacquelyne-veiniest-gilda.ngrok-free.dev
   API Key: secret123

🔗 Available Endpoints:
   GET  https://jacquelyne-veiniest-gilda.ngrok-free.dev/
   POST https://jacquelyne-veiniest-gilda.ngrok-free.dev/generate
   POST https://jacquelyne-veiniest-gilda.ngrok-free.dev/chat
   POST https://jacquelyne-veiniest-gilda.ngrok-free.dev/documentation
   POST https://jacquelyne-veiniest-gilda.ngrok-free.dev/detect-language

💡 Test your API:
   curl 'https://jacquelyne-veiniest-gilda.ngrok-free.dev/'

⏳ Server is running... Keep this cell running!
🛑 To stop: Interrupt the kernel



INFO:     Started server process [47]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:43509 (Press CTRL+C to quit)


INFO:     41.236.163.240:0 - "GET / HTTP/1.1" 200 OK
INFO:     41.236.163.240:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     41.236.163.240:0 - "POST /generate HTTP/1.1" 200 OK

Chat request: Explain the main purpose
Language: python
Code length: 1965 characters

Response generated: 1703 characters
Preview: The main purpose of this Python script is to create and run a two-player number selection game. Here...
INFO:     41.236.163.240:0 - "POST /chat HTTP/1.1" 200 OK
